In [1]:
from utils import prepare_jupyter
prepare_jupyter()

In [16]:
import numpy as np

from structure.Dataset import Dataset
from structure.CatboostEnsemble import CatboostEnsemble
from scipy.special import expit, softmax

In [17]:
train_set, val_set = Dataset.create_cancer().split(0.5)
train_set.X.shape, val_set.X.shape

((284, 30), (285, 30))

In [18]:
params = {
    'loss_function': 'MultiClass' if train_set.name == 'iris' else 'Logloss',
    'n_estimators': 3,
    'verbose': False,
    'depth': 2
}

cb_ensemble = CatboostEnsemble(params)
cb_ensemble.fit(train_set)

len(cb_ensemble.trees)

(4,)
(4,)
(4,)


3

In [21]:
X_test = val_set.X[0:5]
print(cb_ensemble.predict_proba(X_test))
print()
print(cb_ensemble.clf.predict_proba(X_test))

[ 0.72106617 -1.42103886  0.67584519 -0.38529558 -3.42835653]
[[0.32715825 0.67284175]
 [0.80550122 0.19449878]
 [0.33718924 0.66281076]
 [0.59514969 0.40485031]
 [0.96857909 0.03142091]]

[[0.32715825 0.67284175]
 [0.80550122 0.19449878]
 [0.33718924 0.66281076]
 [0.59514969 0.40485031]
 [0.96857909 0.03142091]]


In [5]:
trees = cb_ensemble.trees
clf = cb_ensemble.clf

n_classes = len(clf._classes)
n_estimators = clf.tree_count_

preds = np.array([tree.predict(X_test) for tree in trees])
print(preds)
preds = np.sum(preds, axis=0)
print()
print(preds)

if n_classes > 2:
    results_proba = softmax(preds, axis=1)
else:
    results_proba = np.array([[1 - v, v] for v in expit(preds)])

results_proba

[[-1.63952544 -1.63952544]
 [-0.81419427 -0.81419427]
 [ 0.11216574 -0.92714017]]

[-2.34155397 -3.38085988]


array([[0.91226055, 0.08773945],
       [0.96710097, 0.03289903]])

# Multiclass

In [10]:
train_set, val_set = Dataset.create_iris().split(0.5)

params = {
    'loss_function': 'MultiClass',
    'n_estimators': 3,
    'verbose': False,
    'depth': 2
}

cb_ensemble = CatboostEnsemble(params)
cb_ensemble.fit(train_set)

(12,)
(12,)
(12,)


In [15]:
print(cb_ensemble.predict(val_set.X[:2]))
print()
print(cb_ensemble.clf.predict(val_set.X[:2]))

[1 2]

[[1.]
 [2.]]


In [11]:
trees = cb_ensemble.trees
clf = cb_ensemble.clf

n_classes = len(clf._classes)
n_estimators = clf.tree_count_

preds = np.array([tree.predict(X_test) for tree in trees])
print(preds)
preds = np.sum(preds, axis=0)
print()
print(preds)

if n_classes > 2:
    results_proba = softmax(preds, axis=1)
else:
    results_proba = np.array([[1 - v, v] for v in expit(preds)])

results_proba

[[[-0.01714286  0.          0.039     ]
  [-0.01714286  0.          0.039     ]]

 [[-0.02016872  0.          0.03794936]
  [-0.02016872  0.          0.03794936]]

 [[-0.01837497  0.          0.03166884]
  [-0.01837497  0.          0.03166884]]]

[[-0.05568654  0.          0.10861819]
 [-0.05568654  0.          0.10861819]]


array([[0.3090388 , 0.32673629, 0.36422491],
       [0.3090388 , 0.32673629, 0.36422491]])

In [9]:
n_classes = 3
l = 12 // 3
a = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
a = a.reshape((n_classes, l))
a = np.rollaxis(a, axis=1)
print(a[1::2])
print(a[::2])

[[ 2  6 10]
 [ 4  8 12]]
[[ 1  5  9]
 [ 3  7 11]]
